# Clustering Crypto

In [28]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

### Deliverable 1: Preprocessing the Data for PCA

In [29]:
# Load the crypto_data.csv dataset.
df = pd.read_csv(r'crypto_data.csv', index_col = 0)
df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [30]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = df.loc[df['IsTrading'] == True]
crypto_df = crypto_df.rename(columns={'Unnamed: 0': 'ID'})

# Dropping BitTorrent and 
filter_list = ['BitTorrent', 'Vechain']
crypto_df = crypto_df[~crypto_df.CoinName.isin(filter_list)]


crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [31]:
# Keep all the cryptocurrencies that have a working algorithm.
#crypto_df = crypto_df.loc[df['Algorithm'] != "N/A"]

crypto_df.Algorithm.unique()

#crypto_df.isna().any()

array(['Scrypt', 'X11', 'SHA-256', 'X13', 'Ethash', 'CryptoNight-V7',
       'Equihash', 'SHA-512', 'Multiple', 'X15', 'NIST5', 'Quark',
       'Groestl', 'PoS', 'NeoScrypt', 'SHA3', 'HybridScryptHash256',
       'Scrypt-n', 'PHI1612', 'Lyra2REv2', 'CryptoNight', 'Shabal256',
       'Counterparty', 'Blake', 'Momentum', 'Stanford Folding', 'QuBit',
       'XG Hash', 'M7 POW', 'Curve25519', 'Lyra2RE', 'QUAIT', 'vDPOS',
       'Blake2b', 'BLAKE256', '1GB AES Pattern Search', 'Dagger',
       'CryptoNight-Lite', 'X11GOST', 'SHA-256D', 'POS 3.0',
       'Progressive-n', 'DPoS', 'Lyra2Z', 'X14', 'Time Travel', 'Argon2',
       'Keccak', 'Blake2S', 'Dagger-Hashimoto', '536', 'Argon2d',
       'Cloverhash', 'Skein', 'SkunkHash v2 Raptor', 'Ouroboros',
       'POS 2.0', 'SkunkHash', 'C11', 'Proof-of-BibleHash',
       'SHA-256 + Hive', 'Proof-of-Authority', 'XEVAN', 'VBFT',
       'YescryptR16', 'IMesh', 'Green Protocol', 'Semux BFT consensus',
       'X16R', 'Tribus', 'CryptoNight Heavy', 'Jum

In [32]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop('IsTrading', axis=1)

In [33]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 0 null values
Column TotalCoinSupply has 0 null values


In [34]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[crypto_df ['TotalCoinsMined'] != 0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [35]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = crypto_df['CoinName']
names_df

42          42 Coin
404         404Coin
1337      EliteCoin
BTC         Bitcoin
ETH        Ethereum
           ...     
ZEPH         ZEPHYR
GAP         Gapcoin
BDX          Beldex
ZEN         Horizen
XBC     BitcoinPlus
Name: CoinName, Length: 531, dtype: object

In [36]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop('CoinName', axis=1)
crypto_df


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [37]:
# Use get_dummies() to create variables for text features
x = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
x.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW and PoS,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# Standardize the data with StandardScaler().
x = MinMaxScaler().fit_transform(x)
x.max()

1.0

### Deliverable 2: Reducing Data Dimensions Using PCA

In [49]:
# Using PCA to reduce dimension to three principal components.
pcs_df = PCA(n_components=3)

x = pcs_df.fit_transform(x)

x.max()

0.9469980141741227

In [50]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=x, columns=['PC1','PC2','PC3'],index = crypto_df.index)
pcs_df.head()

# Print the shape of the clustered_df
pcs_df.shape

(531, 3)

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [51]:
# Create an elbow curve to find the best value for K.
# Finding the best value for k
inertia = []
k = list(range(1,10))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=1)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Creating the Elbow Curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Visualization", xlabel="Number of Clusters", ylabel="Inertia")

C:\Users\Chapm\.conda\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1039: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [52]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=1)

# Fit the model
model.fit(x)

# Predict clusters
predictions = model.predict(x)

In [53]:
predictions

array([1, 1, 0, 0, 3, 0, 1, 3, 3, 3, 1, 3, 1, 1, 0, 1, 0, 0, 1, 1, 3, 3,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 3, 1, 0, 0, 0, 0, 1, 0, 3, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 3, 3, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 3, 1, 3, 0, 0, 1, 0, 3, 0, 0, 0, 1, 1, 0, 3, 0, 1,
       0, 1, 0, 0, 1, 3, 1, 3, 1, 1, 0, 0, 1, 0, 3, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 3, 0, 3, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 3, 1, 0, 1, 1, 3, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 0, 3, 3, 1, 1, 1, 1, 0, 1, 3, 1,
       1, 0, 1, 0, 3, 1, 0, 3, 1, 3, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 3, 1, 3, 3, 3, 3, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 3, 1, 3, 3, 0, 1, 0, 1, 0, 1,

In [54]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df, names_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE
clustered_df["Class"] = predictions.tolist()

# Print the shape of the clustered_df
print(clustered_df.shape)

# Print the shape of the clustered_df
clustered_df.shape

(531, 9)


(531, 9)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [55]:
# Creating a 3D-Scatter with the PCA data and the clusters

fig = px.scatter_3d(
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    hover_name = 'CoinName',
    hover_data= ['Algorithm'],
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [47]:
# Create a table with tradable cryptocurrencies.
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
clustered_df.hvplot.table(columns)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [20]:
# Print the total number of tradable cryptocurrencies.
len(clustered_df)

532

In [21]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.

X_scaled = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply","TotalCoinsMined"]])
X_scaled

array([[4.20000000e-11, 3.11497414e-02],
       [5.32000000e-04, 3.67037901e-02],
       [3.14159265e-01, 1.85264300e-01],
       ...,
       [1.40022261e-03, 3.63092201e-02],
       [2.10000000e-05, 3.11881471e-02],
       [1.00000000e-06, 3.11504166e-02]])

In [22]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
x_df = pd.DataFrame(data = X_scaled, 
                  index = crypto_df.index, 
                  columns = ["TotalCoinSupply","TotalCoinsMined"])

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
x_df['CoinName'] = clustered_df.CoinName

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
x_df['Class'] = clustered_df.Class

#plot_df.head(10)

x_df

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.031150,42 Coin,1
404,5.320000e-04,0.036704,404Coin,1
1337,3.141593e-01,0.185264,EliteCoin,1
BTC,2.100000e-05,0.031244,Bitcoin,0
ETH,0.000000e+00,0.031717,Ethereum,0
...,...,...,...,...
ZEPH,2.000000e-03,0.041677,ZEPHYR,1
GAP,2.500000e-04,0.031228,Gapcoin,1
BDX,1.400223e-03,0.036309,Beldex,0
ZEN,2.100000e-05,0.031188,Horizen,0


In [23]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
x_df.hvplot.scatter(x="TotalCoinsMined", 
                     y="TotalCoinSupply",
                     by="Class",
                     xticks="TotalCoinsMined", 
                     title="Scatter Visualization", 
                     xlabel="TotalCoinsMined", 
                     ylabel="TotalCoinSupply")


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)